In [1]:
%pip install tiktoken pymupdf -q

Note: you may need to restart the kernel to use updated packages.


# 필수 패키지 설치

#### src.processors.document_processor.py
HWP를 PDF로 변환하여 처리하는 코드 추가
- LibreOffice + unoconv(pyuno) 변환을 위한 환경 설정 필요

In [ ]:
# -*- coding: utf-8 -*-
import hashlib
import subprocess
import tempfile
import shutil
from pathlib import Path
from typing import Optional, Dict, Any, List, Tuple
import tiktoken
import re 

try:
    import pymupdf
    import pymupdf4llm
    PYMUPDF_AVAILABLE = True
except ImportError:
    PYMUPDF_AVAILABLE = False

from src.db import DocumentsDB
from src.config import get_config
from src.utils.logging_config import get_logger
from tqdm.notebook import tqdm

class DocumentProcessor:
    """
    문서 처리 클래스 (PDF 및 HWP 지원) - 처리 흐름에 따라 함수 재배치

    주요 기능:
    - PDF 파일을 읽어 Markdown으로 변환
    - HWP 파일을 unoconv를 통해 PDF로 변환 후, PDF 처리 로직 재사용
    - 파일 해시 계산 및 토큰 수 계산
    """

    # --- 1. 초기화 및 기본 유틸리티 ---

    def __init__(self, db_path: Optional[str] = None, config=None, progress_callback=None):
        """DocumentProcessor 초기화"""
        self.config = config or get_config()
        self.logger = get_logger(__name__)
        
        if db_path is None:
            db_path = self.config.DOCUMENTS_DB

        self.docs_db = DocumentsDB(db_path)
        self.tokenizer = tiktoken.encoding_for_model(self.config.OPENAI_TOKENIZER_MODEL)
        self.progress_callback = progress_callback

        self.logger.info(f"DocumentProcessor 초기화 완료 (DB: {db_path})")
        if not PYMUPDF_AVAILABLE:
            self.logger.warning("PyMuPDF 라이브러리(pymupdf, pymupdf4llm)가 없어 PDF 및 HWP 처리가 비활성화됩니다.")

    def calculate_file_hash(self, file_path: str) -> str:
        """파일의 SHA-256 해시를 계산합니다."""
        with open(file_path, 'rb') as f:
            return hashlib.sha256(f.read()).hexdigest()

    def count_tokens(self, text: str) -> int:
        """주어진 텍스트의 토큰 수를 계산합니다."""
        return len(self.tokenizer.encode(text))
        
    def clean_markdown_text(self, text: str) -> str:
        """Markdown 텍스트 전처리 (공백, 줄바꿈 정리)"""
        text = re.sub(r'[ \t]+', ' ', text)
        text = re.sub(r'\n{3,}', '\n\n', text)
        lines = [line.strip() for line in text.split('\n')]
        text = '\n'.join(lines)
        return text.strip()

    # --- 2. HWP 변환 헬퍼 (HWP 전용) ---

    def _convert_hwp_to_pdf(self, hwp_path: str) -> Tuple[Optional[str], Optional[str]]:
        """unoconv를 사용하여 HWP를 PDF로 변환합니다."""
        temp_dir = tempfile.mkdtemp()
        pdf_path = Path(temp_dir) / (Path(hwp_path).stem + ".pdf")
        
        cmd = ["unoconv", "-f", "pdf", "-o", str(pdf_path), str(hwp_path)]
        
        try:
            self.logger.debug(f"unoconv 실행, pdf 변환 중: {' '.join(cmd)}")
            subprocess.run(cmd, capture_output=True, text=True, check=True, timeout=60)
            self.logger.info(f"unoconv 변환 성공: {hwp_path} -> {pdf_path}")
            return str(pdf_path), temp_dir

        except FileNotFoundError:
            self.logger.error("unoconv 명령어를 찾을 수 없습니다.")
            shutil.rmtree(temp_dir)
            return None, None
        except subprocess.CalledProcessError as e:
            self.logger.error(f"unoconv 변환 실패: {e.stderr}")
            shutil.rmtree(temp_dir)
            return None, None
        except subprocess.TimeoutExpired:
            self.logger.error("unoconv 변환 시간 초과 (60초).")
            shutil.rmtree(temp_dir)
            return None, None 

    # --- 3. PDF 파싱 및 Markdown 변환 (핵심 기능) ---

    def _markdown_with_progress(self, pdf_path: str, display_file_name: Optional[str] = None) -> List[Dict]:
        """PDF를 페이지별로 Markdown 변환 (진행 상황 표시)"""
        file_name = display_file_name or Path(pdf_path).name
        
        try:
            with pymupdf.open(pdf_path) as doc:
                total_pages = len(doc)
        except Exception as e:
            self.logger.error(f"PDF 파일 열기/페이지 수 확인 실패: {pdf_path}, 오류: {e}")
            return []

        pages_data = []
        with tqdm(total=total_pages, desc="PDF to Markdown", unit="page") as pbar:
            for page_num in range(total_pages):
                pbar_msg = ""
                try:
                    markdown = pymupdf4llm.to_markdown(doc=pdf_path, pages=[page_num])
                    markdown = self.clean_markdown_text(markdown)
                    
                    status = 'processing'
                    if not markdown.strip():
                        status = 'empty'
                        markdown = self.config.EMPTY_PAGE_MARKER
                        pbar_msg = f"빈 페이지: {page_num + 1}"
                    else:
                        pbar_msg = f"페이지 {page_num + 1} len={len(markdown)}"
                    
                    pages_data.append({'page_num': page_num + 1, 'content': markdown})
                    
                    if self.progress_callback:
                        self.progress_callback({
                            'file_name': file_name, 'current_page': page_num + 1,
                            'total_pages': total_pages, 'page_content_length': len(markdown),
                            'status': status, 'error': ""
                        })

                except Exception as e:
                    # 변환 실패 처리
                    pbar_msg = f"페이지 {page_num + 1} 실패: {e}"
                    self.logger.warning(pbar_msg)
                    pages_data.append({'page_num': page_num + 1, 'content': "[변환 실패]"})
                    if self.progress_callback:
                        self.progress_callback({
                            'file_name': file_name, 'current_page': page_num + 1,
                            'total_pages': total_pages, 'page_content_length': 0,
                            'status': 'failed', 'error': str(e)
                        })
                finally:
                    pbar.set_postfix_str(pbar_msg)
                    pbar.update(1)
        return pages_data

    # --- 4. DB 저장 및 통계 계산 (통합 로직) ---

    def _save_processed_data_to_db(self, original_file_path: str, file_hash: str, pages_data: List[Dict], is_hwp_conversion: bool = False) -> bool:
        """변환된 페이지 데이터를 DB에 저장하고 파일 정보를 업데이트합니다."""
        file = Path(original_file_path)
        all_content = []

        # 1. 페이지별 데이터 DB 저장
        for page_data in pages_data:
            page_num = page_data['page_num']
            content = page_data['content']

            is_empty = content == self.config.EMPTY_PAGE_MARKER
            
            if not is_empty and content != "[변환 실패]":
                page_content = f"{self.config.PAGE_MARKER_FORMAT.format(page_num=page_num)}\n\n{content}"
            else:
                page_content = content
            
            all_content.append(page_content)

            token_count = self.count_tokens(page_content)
            self.docs_db.insert_page_data(
                file_hash=file_hash, page_number=page_num, markdown_content=page_content,
                token_count=token_count, is_empty=is_empty
            )

        # 2. 전체 파일 정보 계산 및 DB 저장
        full_content = "\n\n".join(all_content)
        full_content = self.clean_markdown_text(full_content)
        total_tokens = self.count_tokens(full_content)
        total_pages = len(pages_data)

        self.docs_db.insert_file_info(
            file_hash=file_hash, file_name=file.name, total_pages=total_pages, 
            file_size=file.stat().st_size, total_chars=len(full_content), 
            total_tokens=total_tokens
        )
        
        file_type_log = "HWP(PDF 변환)" if is_hwp_conversion else "PDF"
        self.logger.info(
            f"{file_type_log} 처리 완료: {file.name} "
            f"({total_pages} 페이지, {total_tokens:,} 토큰, {file.stat().st_size / 1024:.1f}KB)"
        )
        return True

    # --- 5. 메인 프로세스 (흐름 제어) ---
    
    def _process_pdf(self, pdf_path: str, original_file_path: Optional[str] = None) -> Optional[str]:
        """PDF 파일을 파싱하고 DB에 저장합니다. (HWP 처리에서 재사용됨)"""
        
        original_path = original_file_path or pdf_path
        original_file = Path(original_path)
        
        if not original_file.exists():
            self.logger.error(f"원본 파일 없음: {original_path}")
            return None
        
        file_hash = self.calculate_file_hash(original_path)
        self.logger.debug(f"파일 해시: {file_hash[:16]}...")
        
        self.logger.info(f"PDF 파싱 시작: {Path(pdf_path).name} (원본: {original_file.name})")

        # 3단계: PDF 파싱 실행
        pages_data = self._markdown_with_progress(pdf_path, display_file_name=original_file.name)
        
        # 4단계: DB 저장 실행
        is_hwp = pdf_path != original_path
        success = self._save_processed_data_to_db(
            original_file_path=original_path, 
            file_hash=file_hash, 
            pages_data=pages_data,
            is_hwp_conversion=is_hwp
        )
        
        return file_hash if success else None

    def _process_hwp_as_pdf(self, hwp_path: str) -> Optional[str]:
        """HWP를 PDF로 변환 후, _process_pdf를 사용하여 처리합니다."""
        hwp_file = Path(hwp_path)
        self.logger.info(f"HWP(PDF변환) 처리 시작: {hwp_file.name}")

        temp_dir = None
        file_hash = None
        try:
            # 2단계: HWP -> PDF 변환
            pdf_path, temp_dir = self._convert_hwp_to_pdf(hwp_path)
            
            if not pdf_path:
                if self.progress_callback:
                    self.progress_callback({'file_name': hwp_file.name, 'status': 'failed', 'error': "unoconv 변환 실패"})
                return None

            # 5단계: PDF 파싱 및 DB 저장 로직 재사용
            file_hash = self._process_pdf(pdf_path=pdf_path, original_file_path=hwp_path)
            return file_hash

        except Exception as e:
            self.logger.error(f"HWP 처리 중 예외 발생: {hwp_file.name}, 오류: {e}")
            if self.progress_callback:
                self.progress_callback({'file_name': hwp_file.name, 'status': 'failed', 'error': str(e)})
            return None
        finally:
            # 임시 디렉토리 정리
            if temp_dir:
                try:
                    shutil.rmtree(temp_dir)
                    self.logger.debug(f"임시 디렉토리 삭제: {temp_dir}")
                except Exception as e:
                    self.logger.error(f"임시 디렉토리 삭제 실패: {temp_dir}, 오류: {e}")


    # --- 6. 공통 진입점 ---

    def process_document(self, file_path: str) -> Optional[str]:
        """[공통 진입점] 문서를 처리하고 DB에 저장합니다."""
        file = Path(file_path)
        if not file.exists():
            self.logger.error(f"파일 없음: {file_path}")
            return None

        extension = file.suffix.lower()

        if extension == ".pdf":
            if not PYMUPDF_AVAILABLE:
                self.logger.error("PDF 처리를 위해 'pymupdf'와 'pymupdf4llm' 라이브러리가 필요합니다.")
                return None
            return self._process_pdf(file_path)
        
        elif extension == ".hwp":
            if not PYMUPDF_AVAILABLE: 
                self.logger.error("HWP 처리를 위해 'pymupdf'와 'pymupdf4llm' (및 unoconv)가 필요합니다.")
                return None
            return self._process_hwp_as_pdf(file_path)
        
        else:
            self.logger.warning(f"지원하지 않는 파일 형식: {extension} ({file.name})")
            return None

# document_processer 테스트

In [2]:
# 환경 설정 및 모듈 임포트
import sys
from pathlib import Path

# 프로젝트 루트를 sys.path에 추가
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.processors.document_processor import DocumentProcessor
from src.db.documents_db import DocumentsDB
from src.utils.logging_config import get_logger

logger = get_logger(__name__)
logger.info("테스트 환경 설정 완료")

Consider using the pymupdf_layout package for a greatly improved page layout analysis.
⚠ 설정 검증 실패:
  - MARKDOWN_REMOVE_ELEMENTS에 잘못된 값: {'blockquotes', 'lists'}. 허용: {'html', 'images', 'headers', 'emphasis', 'links'}
2025-11-14 13:50:12 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
2025-11-14 13:50:12 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
2025-11-14 13:50:13 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
2025-11-14 13:50:13 [I] __main__ - 테스트 환경 설정 완료


In [6]:
def test_real_pdf_if_available() -> None:
    """실제 PDF 파일이 있는 경우 테스트"""
    from pathlib import Path
    
    # data/raw 디렉토리에서 PDF 파일 검색
    raw_dir = project_root / "data" / "raw"
    pdf_files = list(raw_dir.glob("*.pdf")) if raw_dir.exists() else []
    
    if not pdf_files:
        logger.warning("⚠ data/raw 디렉토리에 PDF 파일이 없습니다.")
        logger.info("테스트를 건너뜁니다. PDF 파일을 추가하려면:")
        logger.info(f"  경로: {raw_dir}")
        return
    
    logger.info(f"발견된 PDF 파일: {len(pdf_files)}개")
    
    # 첫 번째 PDF 파일 처리
    pdf_path = pdf_files[0]
    logger.info(f"처리 중: {pdf_path.name}")
    
    try:
        processor = DocumentProcessor(db_path=str(project_root / "data" / "test_documents.db"))
        file_hash = processor.process_pdf(str(pdf_path))
        
        if file_hash:
            logger.info(f"✓ PDF 처리 성공: {file_hash[:16]}...")
        else:
            logger.error("✗ PDF 처리 실패 (PyMuPDF 미설치 가능성)")
            
    except Exception as e:
        logger.error(f"✗ PDF 처리 중 오류: {e}")

# 실행
test_real_pdf_if_available()

2025-11-14 14:00:26 [I] __main__ - 발견된 PDF 파일: 100개
2025-11-14 14:00:26 [I] __main__ - 처리 중: 한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf
2025-11-14 14:00:26 [I] [DOCP] - DocumentProcessor 초기화 완료 (DB: /home/spai0438/codeit_ai_g2b_search/data/test_documents.db)
2025-11-14 14:00:26 [I] [DOCP] - PDF 처리 시작: 한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf
2025-11-14 14:00:26 [D] [DOCP] - 파일 해시: 5a28516e24903a44...


PDF to Markdown:   0%|          | 0/96 [00:00<?, ?page/s]

2025-11-14 14:01:45 [I] [DOCP] - PDF 처리 완료: 한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf (96 페이지, 1675.0KB), 변환문자수=116,963
2025-11-14 14:01:45 [I] __main__ - ✓ PDF 처리 성공: 5a28516e24903a44...


## 전체 파일 로드해서 test_documents.db에 넣어보기

In [14]:
def test_real_pdf_if_available() -> None:
    """실제 PDF 파일이 있는 경우 테스트"""
    from pathlib import Path
    from tqdm import tqdm

    # data/raw 디렉토리에서 PDF 파일 검색
    raw_dir = project_root / "data" / "raw"
    pdf_files = list(raw_dir.glob("*.pdf")) if raw_dir.exists() else []
    
    if not pdf_files:
        logger.warning("⚠ data/raw 디렉토리에 PDF 파일이 없습니다.")
        logger.info("테스트를 건너뜁니다. PDF 파일을 추가하려면:")
        logger.info(f"  경로: {raw_dir}")
        return
    
    logger.info(f"발견된 PDF 파일: **{len(pdf_files)}**개")
    
    # DocumentProcessor 인스턴스를 루프 밖에서 한 번만 생성
    try:
        processor = DocumentProcessor(db_path=str(project_root / "data" / "test_documents.db"))
    except Exception as e:
        logger.error(f"✗ DocumentProcessor 초기화 중 오류: {e}")
        return

    # --- 모든 PDF 파일을 처리하는 반복문 구조 ---
    for pdf_path in pdf_files:
        logger.info(f"처리 중: **{pdf_path.name}**")
        
        try:
            file_hash = processor.process_pdf(str(pdf_path))
            
            if file_hash:
                logger.info(f"✓ PDF 처리 성공: {file_hash[:16]}...")
            else:
                # process_pdf가 성공했지만 해시를 반환하지 않은 경우 (예: 파일이 비어 있음)
                logger.error(f"✗ PDF 처리 실패 (파일 해시 반환 안 됨): {pdf_path.name}")
                
        except Exception as e:
            # PyMuPDF 미설치 등 구체적인 오류를 로깅
            logger.error(f"✗ PDF 처리 중 오류 [{pdf_path.name}]: {e}")
            
    logger.info("모든 PDF 파일 처리가 완료되었습니다.")

# 실행
test_real_pdf_if_available()

2025-11-14 14:43:17 [I] __main__ - 발견된 PDF 파일: **100**개
2025-11-14 14:43:17 [I] [DOCP] - DocumentProcessor 초기화 완료 (DB: /home/spai0438/codeit_ai_g2b_search/data/test_documents.db)
2025-11-14 14:43:17 [I] __main__ - 처리 중: **한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf**
2025-11-14 14:43:17 [I] [DOCP] - PDF 처리 시작: 한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf
2025-11-14 14:43:17 [D] [DOCP] - 파일 해시: 5a28516e24903a44...


PDF to Markdown:   0%|          | 0/96 [00:00<?, ?page/s]

2025-11-14 14:44:36 [I] [DOCP] - PDF 처리 완료: 한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf (96 페이지, 1675.0KB), 변환문자수=116,963
2025-11-14 14:44:36 [I] __main__ - ✓ PDF 처리 성공: 5a28516e24903a44...
2025-11-14 14:44:36 [I] __main__ - 처리 중: **광주과학기술원_학사시스템 기능개선 사업.pdf**
2025-11-14 14:44:36 [I] [DOCP] - PDF 처리 시작: 광주과학기술원_학사시스템 기능개선 사업.pdf
2025-11-14 14:44:36 [D] [DOCP] - 파일 해시: 503e8c94230c54f4...


PDF to Markdown:   0%|          | 0/66 [00:00<?, ?page/s]

2025-11-14 14:45:06 [I] [DOCP] - PDF 처리 완료: 광주과학기술원_학사시스템 기능개선 사업.pdf (66 페이지, 1217.7KB), 변환문자수=75,125
2025-11-14 14:45:06 [I] __main__ - ✓ PDF 처리 성공: 503e8c94230c54f4...
2025-11-14 14:45:06 [I] __main__ - 처리 중: **한국지식재산보호원_IP-NAVI  해외지식재산센터 사업관리 시스템 기능개.pdf**
2025-11-14 14:45:06 [I] [DOCP] - PDF 처리 시작: 한국지식재산보호원_IP-NAVI  해외지식재산센터 사업관리 시스템 기능개.pdf
2025-11-14 14:45:06 [D] [DOCP] - 파일 해시: b9cab07e470a3d15...


...

2025-11-14 16:31:26 [I] [DOCP] - PDF 처리 완료: 서울특별시 여성가족재단_(재공고, 협상) 서울 디지털성범죄 안심지원센.pdf (141 페이지, 2619.9KB), 변환문자수=145,387
2025-11-14 16:31:26 [I] __main__ - ✓ PDF 처리 성공: 15b18199a85b44a0...
2025-11-14 16:31:26 [I] __main__ - 처리 중: **재단법인경기도일자리재단_2025년 통합접수시스템 운영.pdf**
2025-11-14 16:31:26 [I] [DOCP] - PDF 처리 시작: 재단법인경기도일자리재단_2025년 통합접수시스템 운영.pdf
2025-11-14 16:31:26 [D] [DOCP] - 파일 해시: bb60a1a2e26b8e28...


PDF to Markdown:   0%|          | 0/137 [00:00<?, ?page/s]

2025-11-14 16:33:05 [I] [DOCP] - PDF 처리 완료: 재단법인경기도일자리재단_2025년 통합접수시스템 운영.pdf (137 페이지, 2034.2KB), 변환문자수=118,153
2025-11-14 16:33:05 [I] __main__ - ✓ PDF 처리 성공: bb60a1a2e26b8e28...
2025-11-14 16:33:05 [I] __main__ - 처리 중: **한국전기안전공사_전기안전 관제시스템 보안 모듈 개발 용역.pdf**
2025-11-14 16:33:05 [I] [DOCP] - PDF 처리 시작: 한국전기안전공사_전기안전 관제시스템 보안 모듈 개발 용역.pdf
2025-11-14 16:33:05 [D] [DOCP] - 파일 해시: 41e3b5ed9763d68c...


PDF to Markdown:   0%|          | 0/74 [00:00<?, ?page/s]

2025-11-14 16:33:43 [I] [DOCP] - PDF 처리 완료: 한국전기안전공사_전기안전 관제시스템 보안 모듈 개발 용역.pdf (74 페이지, 1698.9KB), 변환문자수=61,202
2025-11-14 16:33:43 [I] __main__ - ✓ PDF 처리 성공: 41e3b5ed9763d68c...
2025-11-14 16:33:43 [I] __main__ - 모든 PDF 파일 처리가 완료되었습니다.


## 5. 실제 PDF 파일 테스트 (선택사항)

In [12]:
def verify_database_content() -> None:
    """DB 내용 확인 및 검증"""
    import sqlite3
    
    test_db_path = project_root / "data" / "test_documents.db"
    
    with sqlite3.connect(test_db_path) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        # 1. 파일 정보 조회
        # 모든 테이블 이름 조회 쿼리
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        logger.info(f"모든 테이블 이름 조회 쿼리 {cursor.fetchall()}")
        files = cursor.fetchall()
        logger.info(f"\n=== 파일 정보 ({len(files)}건) ===")
        for row in files:
            logger.info(f"  - {row['file_name']}: {row['total_pages']}페이지, {row['total_tokens']}토큰")
        
        # 2. 페이지 데이터 조회
        # cursor.execute("""
        #     SELECT file_hash, page_number, is_empty 
        #     FROM page_data 
        #     ORDER BY file_hash, page_number
        # """)
        pages = cursor.fetchall()
        logger.info(f"\n=== 페이지 데이터 ({len(pages)}건) ===")
        
        # 파일별로 그룹화
        from collections import defaultdict
        pages_by_file = defaultdict(list)
        for row in pages:
            pages_by_file[row['file_hash']].append(row)
        
        for file_hash, file_pages in pages_by_file.items():
            logger.info(f"  파일: {file_hash[:16]}...")
            for page in file_pages:
                status = "빈페이지" if page['is_empty'] else "정상"
                logger.info(f"    - 페이지 {page['page_number']}: {page['token_count']}토큰 ({status})")
        
        # 3. 통계 조회
        db = DocumentsDB(str(test_db_path))
        stats = db.get_document_stats()
        logger.info(f"\n=== 전체 통계 ===")
        logger.info(f"  총 파일: {stats['total_files']}개")
        logger.info(f"  총 페이지: {stats['total_pages']}개")
        # logger.info(f"  총 토큰: {stats['total_tokens']:,}개")
        logger.info(f"  총 크기: {stats['total_size_mb']}MB")
        
        logger.info("\n✓ DB 검증 완료")

# 실행
verify_database_content()

2025-11-14 14:20:49 [I] __main__ - 모든 테이블 이름 조회 쿼리 [<sqlite3.Row object at 0x7a2692c3f6a0>]
2025-11-14 14:20:49 [I] __main__ - 
=== 파일 정보 (0건) ===
2025-11-14 14:20:49 [I] __main__ - 
=== 페이지 데이터 (0건) ===
2025-11-14 14:20:49 [I] __main__ - 
=== 전체 통계 ===
2025-11-14 14:20:49 [I] __main__ -   총 파일: 1개
2025-11-14 14:20:49 [I] __main__ -   총 페이지: 96개
2025-11-14 14:20:49 [I] __main__ -   총 크기: 1.64MB
2025-11-14 14:20:49 [I] __main__ - 
✓ DB 검증 완료


## 4. DB 조회 및 검증

In [9]:
def test_pdf_processing_simulation() -> None:
    """PDF 처리 로직 시뮬레이션 (PyMuPDF 없이)"""
    
    # 실제 PDF 대신 더미 데이터로 로직 테스트
    test_db_path = project_root / "data" / "test_documents.db"
    processor = DocumentProcessor(db_path=str(test_db_path))
    
    # PDF 페이지 시뮬레이션
    dummy_pages = [
        "# 제목 페이지\n\n이것은 첫 번째 페이지입니다.",
        "# 본문 1\n\n두 번째 페이지 내용입니다.",
        "",  # 빈 페이지
        "# 결론\n\n마지막 페이지입니다."
    ]
    
    # 파일 정보 시뮬레이션
    dummy_hash = "dummy_pdf_hash_67890"
    dummy_filename = "test_dummy.pdf"
    
    logger.info(f"PDF 처리 시뮬레이션: {dummy_filename}")
    
    # 페이지별 처리
    all_content = []
    for page_num, page_text in enumerate(dummy_pages, start=1):
        is_empty = len(page_text.strip()) < 10
        
        if is_empty:
            page_content = "[빈 페이지]"
        else:
            page_content = f"--- 페이지 {page_num} ---\n\n{page_text}"
        
        all_content.append(page_content)
        
        # DB 저장
        token_count = processor.count_tokens(page_content)
        processor.docs_db.insert_page_data(
            file_hash=dummy_hash,
            page_number=page_num,
            markdown_content=page_content,
            token_count=token_count,
            is_empty=is_empty
        )
        logger.info(f"페이지 {page_num}: {token_count} 토큰, 빈페이지={is_empty}")
    
    # 전체 콘텐츠
    full_content = "\n\n".join(all_content)
    total_tokens = processor.count_tokens(full_content)
    
    # 파일 정보 저장
    processor.docs_db.insert_file_info(
        file_hash=dummy_hash,
        file_name=dummy_filename,
        total_pages=len(dummy_pages),
        file_size=len(full_content.encode('utf-8')),
        total_chars=len(full_content),
        total_tokens=total_tokens
    )
    
    logger.info(f"✓ PDF 시뮬레이션 완료: {len(dummy_pages)}페이지, {total_tokens}토큰")

# 테스트 실행
test_pdf_processing_simulation()

2025-11-08 22:36:33 [I] src.processors.document_processor - DocumentProcessor 초기화 완료 (DB: d:\GoogleDrive\codeit_ai_g2b_search\data\test_documents.db)
2025-11-08 22:36:33 [I] __main__ - PDF 처리 시뮬레이션: test_dummy.pdf
2025-11-08 22:36:33 [I] __main__ - PDF 처리 시뮬레이션: test_dummy.pdf
2025-11-08 22:36:33 [I] __main__ - 페이지 1: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 2: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 1: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 2: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 3: 6 토큰, 빈페이지=True
2025-11-08 22:36:33 [I] __main__ - 페이지 4: 19 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 3: 6 토큰, 빈페이지=True
2025-11-08 22:36:33 [I] __main__ - 페이지 4: 19 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - ✓ PDF 시뮬레이션 완료: 4페이지, 69토큰
2025-11-08 22:36:33 [I] __main__ - ✓ PDF 시뮬레이션 완료: 4페이지, 69토큰


## 3. PDF 처리 테스트 (PyMuPDF 없이 시뮬레이션)

In [8]:
def test_document_processor_utils() -> None:
    """DocumentProcessor 유틸리티 함수 테스트"""
    import tempfile
    
    # DocumentProcessor 초기화
    test_db_path = project_root / "data" / "test_documents.db"
    processor = DocumentProcessor(db_path=str(test_db_path))
    
    # 1. 임시 파일 생성
    with tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=False, encoding='utf-8') as tf:
        test_content = "This is a test document for hash calculation."
        tf.write(test_content)
        temp_file_path = Path(tf.name)
    
    try:
        # 2. 파일 해시 계산
        file_hash = processor.calculate_file_hash(temp_file_path)
        logger.info(f"파일 해시: {file_hash[:16]}...")
        assert len(file_hash) == 64, "SHA-256 해시 길이 오류"
        
        # 3. 토큰 카운트
        token_count = processor.count_tokens(test_content)
        logger.info(f"토큰 수: {token_count}")
        assert token_count > 0, "토큰 카운트 오류"
        
        logger.info("✓ 유틸리티 함수 테스트 성공")
        
    finally:
        # 임시 파일 삭제
        if temp_file_path.exists():
            temp_file_path.unlink()

# 테스트 실행
test_document_processor_utils()

2025-11-08 22:36:26 [I] src.processors.document_processor - DocumentProcessor 초기화 완료 (DB: d:\GoogleDrive\codeit_ai_g2b_search\data\test_documents.db)
2025-11-08 22:36:26 [I] __main__ - 파일 해시: 258983484b09a700...
2025-11-08 22:36:26 [I] __main__ - 토큰 수: 9
2025-11-08 22:36:26 [I] __main__ - ✓ 유틸리티 함수 테스트 성공
2025-11-08 22:36:26 [I] __main__ - 파일 해시: 258983484b09a700...
2025-11-08 22:36:26 [I] __main__ - 토큰 수: 9
2025-11-08 22:36:26 [I] __main__ - ✓ 유틸리티 함수 테스트 성공


## 2. DocumentProcessor 해시 계산 및 토큰 카운트 테스트

In [7]:
def test_documents_db_crud() -> None:
    """DocumentsDB CRUD 테스트"""
    from datetime import datetime
    import sqlite3
    
    # 테스트용 DB 경로
    test_db_path = project_root / "data" / "test_documents.db"
    test_db_path.parent.mkdir(parents=True, exist_ok=True)
    
    # DB 초기화
    db = DocumentsDB(str(test_db_path))
    
    # 기존 테스트 데이터 삭제 (파일 삭제 대신)
    with sqlite3.connect(test_db_path) as conn:
        conn.execute("DELETE FROM page_data")
        conn.execute("DELETE FROM file_info")
        conn.commit()
    
    logger.info(f"테스트 DB 초기화: {test_db_path}")
    
    # 1. 파일 정보 삽입 (Create)
    test_hash = "test_hash_12345"
    success = db.insert_file_info(
        file_hash=test_hash,
        file_name="test_document.pdf",
        total_pages=3,
        file_size=1024 * 50,  # 50KB
        total_chars=5000,
        total_tokens=1200
    )
    logger.info(f"파일 정보 삽입: {success}")
    
    # 2. 페이지 데이터 삽입
    for page_num in range(1, 4):
        content = f"# 페이지 {page_num}\n\n테스트 콘텐츠입니다."
        db.insert_page_data(
            file_hash=test_hash,
            page_number=page_num,
            markdown_content=content,
            token_count=100 + page_num * 10,
            is_empty=False
        )
    logger.info("페이지 데이터 삽입 완료")
    
    # 3. 통계 조회 (Read)
    stats = db.get_document_stats()
    logger.info(f"문서 통계: {stats}")
    
    # 4. 데이터 확인
    assert stats['total_files'] >= 1, "파일 개수 불일치"
    assert stats['total_pages'] >= 3, "페이지 개수 불일치"
    
    logger.info("✓ CRUD 테스트 성공")
    return db, test_hash

# 테스트 실행
db, test_hash = test_documents_db_crud()

2025-11-08 22:36:20 [I] __main__ - 테스트 DB 초기화: d:\GoogleDrive\codeit_ai_g2b_search\data\test_documents.db
2025-11-08 22:36:20 [I] __main__ - 파일 정보 삽입: True
2025-11-08 22:36:20 [I] __main__ - 파일 정보 삽입: True
2025-11-08 22:36:20 [I] __main__ - 페이지 데이터 삽입 완료
2025-11-08 22:36:20 [I] __main__ - 문서 통계: {'total_files': 1, 'total_pages': 3, 'total_tokens': 1200, 'total_size_bytes': 51200, 'total_size_mb': 0.05}
2025-11-08 22:36:20 [I] __main__ - ✓ CRUD 테스트 성공
2025-11-08 22:36:20 [I] __main__ - 페이지 데이터 삽입 완료
2025-11-08 22:36:20 [I] __main__ - 문서 통계: {'total_files': 1, 'total_pages': 3, 'total_tokens': 1200, 'total_size_bytes': 51200, 'total_size_mb': 0.05}
2025-11-08 22:36:20 [I] __main__ - ✓ CRUD 테스트 성공


## 1. DocumentsDB 기본 CRUD 테스트

## 테스트 요약

위의 테스트들을 통해 다음을 확인했습니다:

1. **DocumentsDB CRUD**: 파일 정보/페이지 데이터 삽입 및 조회 정상 동작
2. **DocumentProcessor 유틸리티**: 파일 해시 계산, 토큰 카운트 정상
3. **PDF 처리 시뮬레이션**: 페이지별 콘텐츠 처리 로직 검증
4. **DB 검증**: 전체 통계 및 데이터 무결성 확인
5. **실제 PDF 처리**: PyMuPDF 사용 가능 시 자동 처리

**추가 테스트 방법**:
- `data/raw/` 디렉토리에 PDF 파일을 추가하면 실제 PDF 처리 테스트 자동 실행됩니다.
- HWP 파일 처리는 현재 `document_processor.py`에 구현되지 않았으므로 향후 추가 필요합니다.